# **Библиотеки**

In [1]:
import librosa
import zipfile
import io
from scipy.io import wavfile
from google.colab import drive
import numpy as np
import pandas as pd

# **Данные**

# Подгрузка данных

In [2]:
drive.mount('/content/drive')
df=pd.read_csv('drive/MyDrive/Анализ_акустических_событий/train.csv', sep=',',header=None)
dataset = zipfile.ZipFile('drive/MyDrive/Анализ_акустических_событий/audio_train.zip')
labels = df.values.tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Создание словарей с переводом label названий в label цифры и наоборот

In [3]:
label_to_figure = {}
count = 0
for i in range(len(labels)):
    if i == 0:
        pass
    elif labels[i][1] in label_to_figure:
        pass
    else:
        label_to_figure[labels[i][1]] = count
        count += 1

figure_to_label = {}
for i in label_to_figure.items():
    figure_to_label[i[1]] = i[0]

# Достаём wav файлы из ZIP

In [4]:
def read_from_zip(filename):
    with io.BytesIO() as bytes_buff:
        bytes_data = dataset.read(filename)
        bytes_buff.write(bytes_data)
        bytes_buff.seek(0)
        sr, wave = wavfile.read(bytes_buff)
        mel = librosa.feature.melspectrogram(wave, sr)
    return sr, mel

# Создаём Mel признаки

In [5]:
!nvidia-smi

Thu Jun 17 06:58:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import pandas as pd
X = []
Y = []
for i in range(len(labels)//2, len(labels)):
    if not i == 0:
        sr, X_mel = read_from_zip(f'train/{labels[i][0]}')
        # vre = np.zeros(480000)
        # vre[:X_mel.shape[0]] = X_mel
        X.append(X_mel)
        Y.append(label_to_figure[labels[i][1]])
# X = np.array(X)
# Y = np.array(Y)

In [7]:
les = []
for i in range(len(X)):
    le = []
    for j in range(len(X[i])):
        for p in range(len(X[i][j])):
            le.append(X[i][j][p])
    le = np.array(le)
    vre = np.zeros(120064)
    vre[:le.shape[0]] = le
    les.append(vre)
X = np.array(les)
Y = np.array(Y)

# **Сохраняем pсl**

Не получается, прерывается из-за ОЗУ

In [ ]:
import pickle

output = open('X.pkl', 'wb')
pickle.dump(X, output)
output.close()

output = open('Y.pkl', 'wb')
pickle.dump(Y, output)
output.close()

pkl_file = open('X.pkl', 'rb')
X = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('Y.pkl', 'rb')
Y = pickle.load(pkl_file)
pkl_file.close()

# **Классификатор**

# Делим данные на траин и валидацию

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

# Обучение

In [10]:
from sklearn.svm import LinearSVC
model = LinearSVC(random_state = 51, C = 1)
model.fit(X_train, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=51, tol=0.0001,
          verbose=0)

# Тест

# Средний f1 = 0.1
Датасет делил на две части, f1.1 = 0.9 и f1.2 = 0.11

# Вторая половина базы

In [11]:
predictions = model.predict(X_test)

In [12]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='macro'))
print(classification_report(y_test, predictions))

0.07288835704668706
              precision    recall  f1-score   support

           0       0.22      0.62      0.33        13
           1       0.00      0.00      0.00        16
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00        11
           4       0.40      0.07      0.12        28
           5       0.03      0.04      0.03        24
           6       0.03      0.09      0.05        11
           7       0.00      0.00      0.00        23
           8       0.00      0.00      0.00        29
           9       0.06      0.17      0.08        23
          10       0.29      0.44      0.35        27
          11       0.09      0.03      0.05        33
          12       0.17      0.04      0.07        24
          13       0.00      0.00      0.00        18
          14       0.43      0.75      0.55        12
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00        28
       

# Первая половина базы

In [11]:
predictions = model.predict(X_test)

In [12]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='macro'))
print(classification_report(y_test, predictions))

0.08490946009874928
              precision    recall  f1-score   support

           0       0.27      0.33      0.30        18
           1       0.08      0.21      0.12        29
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00        17
           4       0.22      0.08      0.12        24
           5       0.04      0.06      0.05        33
           6       0.00      0.00      0.00         8
           7       0.33      0.12      0.17        26
           8       0.09      0.05      0.06        22
           9       0.07      0.09      0.08        32
          10       0.25      0.52      0.34        33
          11       0.22      0.07      0.11        27
          12       0.00      0.00      0.00        22
          13       0.00      0.00      0.00        31
          14       0.61      0.48      0.54        23
          15       0.00      0.00      0.00         8
          16       0.14      0.06      0.09        32
       

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
